<a href="https://colab.research.google.com/github/xloem/techsketball/blob/main/model_import_sketch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

starting_model_path = 't5-base'#'bigscience/T0pp'
input_width = 512


#!pip install jax[tpu] -f https://storage.googleapis.com/jax-releases/libtpu_releases.html
!pip3 install deepspeed
!pip3 install transformers
!pip3 install flax
!pip3 install sentencepiece
!git clone https://github.com/xloem/techsketball && ln -s techsketball/* .

     |████████████████████████████████| 517 kB 5.4 MB/s 
     |████████████████████████████████| 108 kB 60.2 MB/s 
     |████████████████████████████████| 54 kB 1.9 MB/s 
     |████████████████████████████████| 18.2 MB 126 kB/s 
  Created wheel for deepspeed: filename=deepspeed-0.5.8-py3-none-any.whl size=532129 sha256=1c991ecae1c7b9b5264261417b7769e66591e33084ec0d10873bd8c0c5818970
  Stored in directory: /root/.cache/pip/wheels/8d/c6/82/cabd9a300c582a221591fd2c8c997e1f03f601e748aad44e4e
Successfully built deepspeed
     |████████████████████████████████| 3.4 MB 5.4 MB/s 
     |████████████████████████████████| 61 kB 396 kB/s 
     |████████████████████████████████| 895 kB 52.6 MB/s 
     |████████████████████████████████| 3.3 MB 40.8 MB/s 
     |████████████████████████████████| 596 kB 41.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 20

In [2]:
import jax.tools.colab_tpu
jax.tools.colab_tpu.setup_tpu()
jax.local_devices()

In [3]:
from transformers import T5Tokenizer, FlaxT5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained(starting_model_path) # only for source, not for binary
model = FlaxT5ForConditionalGeneration.from_pretrained(starting_model_path)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

INFO:absl:Unable to initialize backend 'gpu': FAILED_PRECONDITION: No visible GPU devices.
INFO:absl:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.


In [4]:
from find_pycode import pair_finder
print('generating training data ...')
data_tuples = [
  (bytes, str)
  for bytes, str
  in pair_finder(globals()) # finds code examples
  if len(bytes) <= input_width
]

/usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py:171: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  "torch.distributed.reduce_op is deprecated, please use "


In [5]:
import jax, numpy as np
data_input_ids = jax.numpy.stack([
  np.frombuffer(bytes.ljust(input_width, b'\0'), dtype=np.uint8)
  for bytes, str in data_tuples
])
data_labels = [str for bytes, str in data_tuples]


In [6]:
#import deepspeed
#cmd_args = None
#model_engine, optimizer, _, _ = deepspeed.initialize(args=cmd_args,
#                                                     model=model,
#                                                     model_parameters=params)